Use Neural Networks to Classify Product for Otto Group 

1. まずコンペのDescriptionを読みましょう
1. Descriptionからどういう問題なのか、回帰・分類などなどを判別しましょう
  - [Otto Group Product Classification Challenge](https://www.kaggle.com/c/otto-group-product-classification-challenge)
> For this competition, we have provided a dataset with 93 features for more than 200,000 products. **The objective is to build a predictive model which is able to distinguish between our main product categories.** The winning models will be open sourced.
  - [Titanic: Machine Learning from Disaster](https://www.kaggle.com/c/titanic)
  > In this challenge, **we ask you to complete the analysis of what sorts of people were likely to survive**. In particular, we ask you to apply the tools of machine learning to predict which passengers survived the tragedy.
  - [New York City Taxi Fare Prediction](https://www.kaggle.com/c/new-york-city-taxi-fare-prediction)
  > While you can get a** basic estimate based on just the distance between the two points** this will result in an RMSE of $5-$8, 
1. さらにコンペの**Data Description**と**Evaluation**を見てみましょう
  - [Otto Group Product Classification Challenge](https://www.kaggle.com/c/otto-group-product-classification-challenge)
  > Submissions are evaluated using the multi-class logarithmic loss.** Each product has been labeled with one true category**. For each product, you must submit a set of predicted **probabilities (one for every category)**.
  - [Titanic: Machine Learning from Disaster](https://www.kaggle.com/c/titanic)
  > It is your job to predict if a passenger survived the sinking of the Titanic or not. 
For each PassengerId in the test set, **you must predict a 0 or 1 value for the Survived variable**.
  - [New York City Taxi Fare Prediction](https://www.kaggle.com/c/new-york-city-taxi-fare-prediction)
  > While you can get a** basic estimate based on just the distance between the two points** this will result in an RMSE of $5-$8, 

### Data Description
- train.csv
  - モデルの学習に使うデータ
- test.csv
  - モデルの予測に使うデータ
- sampleSubmission.csv
  - 結果を提出する際のフォーマット用のデータ

### Evaluation
リーダーボードで表示されるスコアをどのように計算するか、計算するための評価関数
- [Gunosy - いまさら聞けない機械学習の評価関数](https://data.gunosy.io/entry/2016/08/05/115345)

![](https://cdn-ak.f.st-hatena.com/images/fotolife/d/dr_paradi/20160804/20160804201109.png)
> 上図の場合には、予測モデル2の評価が高くなります(Multi-class logarithmic loss自体は0に近い方がよい)。
Accuracyで評価した場合には予測モデル1、予測モデル2の双方とも同じ評価になります。

In [ ]:
import numpy as np
import pandas as pd

CSVの読み込みはpandasというライブラリを使用します
- [pandas](https://pandas.pydata.org/)

pandasはCSVファイルを読み込んで取り扱いやすい表データに変換してくれます
とりあえず、kaggleのCSV読み込み=pandasを利用して読み込むと考えてもらって結構です

In [ ]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
submit = pd.read_csv("../input/sampleSubmission.csv")

In [ ]:
train.head()

In [ ]:
train.describe()

In [ ]:
train.dtypes

In [ ]:
test.head()

In [ ]:
submit.head()

In [ ]:
# 訓練データ 特徴名取り出し
columns = train.columns[1:-1]
# idとtarget列を取り除いたものをXとする
X = train[columns]
print(columns)

In [ ]:
X.head()

In [ ]:
# 訓練データのラベル部分
target = train['target']
target.head()

In [ ]:
# 予測されるラベル名の取り出し
# 上記targetは2次元データであるため1次元のデータに変換する
# https://deepage.net/features/numpy-ravel.html
y = np.ravel(target)
print(y)

In [ ]:
# yの重複なしを調べる
labels = np.unique(y)
labels

In [ ]:
# 訓練データは 61878個の種類があり、それぞれ93個の特徴量を持っている
X.shape

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
# 
model = MLPClassifier(solver= 'lbfgs', hidden_layer_sizes= (30, 10), alpha= 1e-5, random_state=1)
# 用意した訓練データ Xと出力ラベル yを学習する 
model.fit(X, target)

![](https://www.tel.co.jp/museum/magazine/communication/160229_report01_02/img/img_report01_03.jpg)

In [ ]:
# 訓練データと同じく特徴量93種類のみ取り出す
_test = test[columns]
_test.head()

In [ ]:
# テストデータは 144368個の種類があり、それぞれ93個の特徴量を持っている
_test.shape

In [ ]:
# テストデータを使ってどの出力ラベルになりうるか予測する
# predict_probaは出力ラベルそれぞれの確立を吐き出す
test_prob = model.predict_proba(_test)
test_prob

In [ ]:
test_prob.shape

In [ ]:
# 試しにあるデータの出力結果を見てみる
# 出力ラベルは 9種類のラベル
print(labels)
# それぞれ9種類のどれにあたるか、それぞれの確立が出力にあたる
print(test_prob[0])
# 今回の場合 class4 が予測結果になる
print("Class_{}".format(np.argmax(test_prob[0])+1))

In [ ]:
submit[labels] = test_prob

In [ ]:
submit.head()

In [ ]:
submit.to_csv('./otto_prediction.csv', index = False)